In [1]:
# Import des librairies
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics 
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.metrics as metrics
import math
from sklearn.datasets import load_iris

In [2]:
# Chargement du dataset
data, target = load_iris(return_X_y=True, as_frame=True)

In [3]:
# Récupération de la cible dans le dataframe "data"
data['target'] = target

In [4]:
# Shuffle pour désorganiser l'ordre des cibles
data = data.sample(frac=1).reset_index()

In [5]:
# Définition de la cible et des features
features = ['petal length (cm)', 'petal width (cm)']
y = data.target
X = data[features]

In [6]:
# Définition de la taille des jeux
test_size = 0.2
valid_size = 0.2
train_size = 0.6
idx_max = X.index.max()

In [7]:
# Conversion en array numpy
X_numpy = X.to_numpy()
y_numpy = y.to_numpy()

In [8]:
# Split pour X, méthode déterministe, pas de shuffle ou autre
x_sets = np.split(X_numpy, [math.floor(train_size * idx_max),
                           math.floor(train_size * idx_max + valid_size * idx_max),
                           idx_max])

X_train = x_sets[0]
X_valid = x_sets[1]
X_test = x_sets[2]

In [9]:
# De même pour y
y_sets = np.split(y_numpy, [math.floor(train_size * idx_max),
                           math.floor(train_size * idx_max + valid_size * idx_max),
                           idx_max])

y_train = y_sets[0]
y_valid = y_sets[1]
y_test = y_sets[2]

In [10]:
# Prédictions sur le jeu de validation
max_depths = [i for i in range(3, 11)]
scores_valid = {}
scores_train = {}

In [11]:
for max_depth in max_depths:
    # Définition du modèle
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

    # Fit sur l'ensemble d'entraînement
    model.fit(X_train, y_train)

    # Prédictions sur l'ensemble d'entraînement
    pred_train = model.predict(X_train)
    scoring_train = metrics.accuracy_score(y_train, pred_train)
    scores_train[str(max_depth)] = scoring_train

    # Prédictions sur l'ensemble de validation
    pred_valid = model.predict(X_valid)
    scoring_valid = metrics.accuracy_score(y_valid, pred_valid)
    scores_valid[str(max_depth)] = scoring_valid

In [12]:
print("Scores sur le jeu d'entraînement:", scores_train)
print("Scores sur le jeu de validation:", scores_valid)

Scores sur le jeu d'entraînement: {'3': 0.9775280898876404, '4': 0.9887640449438202, '5': 1.0, '6': 1.0, '7': 1.0, '8': 1.0, '9': 1.0, '10': 1.0}
Scores sur le jeu de validation: {'3': 0.9333333333333333, '4': 0.9333333333333333, '5': 0.9333333333333333, '6': 0.9333333333333333, '7': 0.9333333333333333, '8': 0.9333333333333333, '9': 0.9333333333333333, '10': 0.9333333333333333}


In [13]:
# Prédictions sur le jeu de test
max_depth = 3
pred = model.predict(X_test)
scoring = metrics.accuracy_score(y_test, pred)
print(scoring)

0.9333333333333333


In [14]:
# Fonction de découpage en ensembles pour la validation croisée
def k_split(X_trainvalid, y_trainvalid, k, loo, train_size=0.8):
    # Récupère l'indice maximum de X_trainvalid
    idx_max = X_trainvalid.index.max()

    # Définit les points de séparation pour les ensembles
    if loo:
        bouts = [i for i in range(0, X_trainvalid.index.max(), 1)]
    else:
        bouts = [math.floor(i) for i in np.linspace(0, idx_max + 1, k + 1)]

    # Initialise une liste pour stocker les ensembles découpés
    sets = []

    # Itère sur le nombre de plis (k)
    i = 0
    while i < k:
        # Définit les indices de début et de fin pour le pli actuel
        i_bot = bouts[i]
        i_sup = bouts[i + 1]
        indexes = np.arange(i_bot, i_sup, 1)

        # Découpe X_trainvalid et y_trainvalid en ensembles de validation et d'entraînement
        X_valid = X_trainvalid.loc[X_trainvalid.index.isin(indexes)].to_numpy()
        y_valid = y_trainvalid.loc[y_trainvalid.index.isin(indexes)].to_numpy()

        y_train = y_trainvalid.loc[~y_trainvalid.index.isin(indexes)].to_numpy()
        X_train = X_trainvalid.loc[~X_trainvalid.index.isin(indexes)].to_numpy()

        # Ajoute les ensembles découpés à la liste
        sets.append([X_train, X_valid, y_train, y_valid])
        i += 1

    # Retourne la liste des ensembles découpés
    return sets

In [15]:
# Fonction de validation croisée k-fold
def kfold(X, y, model, k=5, test_size=0.2, loo=False):
    # Initialise un dictionnaire pour stocker les scores
    scores = {}

    # Sépare les données en ensemble de test et ensemble d'entraînement + validation
    X_test_trainvalid = np.split(X, [math.floor(X.index.shape[0] * (1 - test_size)), X.index.shape[0]])
    y_test_trainvalid = np.split(y, [math.floor(y.index.shape[0] * (1 - test_size)), y.index.shape[0]])

    # Récupère les données du jeu de test
    X_test = X_test_trainvalid[1].to_numpy()
    y_test = y_test_trainvalid[1].to_numpy()

    # Récupère les données du jeu d'entraînement + validation
    X_trainvalid_full = X_test_trainvalid[0]
    y_trainvalid_full = y_test_trainvalid[0]

    # Utilise la fonction k_split pour découper les données en k ensembles
    sets = k_split(X_trainvalid_full, y_trainvalid_full, k, loo)

    # Initialise des listes pour stocker les scores d'entraînement, de validation et de test
    score_train = []
    score_valid = []
    score_test = []

    # Itère sur chaque ensemble de la validation croisée k-fold
    for X_train, X_valid, y_train, y_valid in sets:
        # Entraîne le modèle sur l'ensemble d'entraînement
        model.fit(X_train, y_train)

        # Prédiction sur l'ensemble d'entraînement
        pred_train = model.predict(X_train)

        # Prédiction sur l'ensemble de validation
        pred_valid = model.predict(X_valid)

        # Concatène les ensembles d'entraînement et de validation
        X_trainvalid = np.concatenate([X_train, X_valid])
        y_trainvalid = np.concatenate([y_train, y_valid])

        # Ré-entraîne le modèle sur l'ensemble d'entraînement + validation
        model.fit(X_trainvalid, y_trainvalid)

        # Prédiction sur l'ensemble de test
        pred_test = model.predict(X_test)

        # Calcule et stocke les scores d'accuracy pour chaque ensemble
        score_train.append(metrics.accuracy_score(y_train, pred_train))
        score_valid.append(metrics.accuracy_score(y_valid, pred_valid))
        score_test.append(metrics.accuracy_score(y_test, pred_test))

    # Calcule les moyennes des scores d'accuracy pour chaque ensemble
    scores['train'] = np.array(score_train).mean()
    scores['valid'] = np.array(score_valid).mean()
    scores['test'] = np.array(score_test).mean()

    # Retourne le dictionnaire des scores
    return scores

In [16]:
# Liste des valeurs de profondeur maximale à tester
max_depths = [i for i in range(3, 11)]

# Nombre de plis pour la validation croisée k-fold
k = 4

# Dictionnaire pour stocker les scores de chaque profondeur maximale
scores = {}

# Itère sur chaque profondeur maximale à tester
for max_depth in max_depths:
    # Crée un modèle DecisionTreeClassifier avec la profondeur maximale spécifiée
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

    # Applique la fonction kfold pour effectuer la validation croisée
    # et récupère les scores d'accuracy pour chaque ensemble (train, valid, test)
    scoring = kfold(X, y, model, k)

    # Stocke les scores dans le dictionnaire avec la profondeur maximale comme clé
    scores[str(max_depth)] = scoring

# Affiche les scores pour chaque profondeur maximale
print(scores)

{'3': {'train': 0.9777777777777777, 'valid': 0.95, 'test': 0.9666666666666667}, '4': {'train': 0.9805555555555555, 'valid': 0.95, 'test': 0.9666666666666667}, '5': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}, '6': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}, '7': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}, '8': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}, '9': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}, '10': {'train': 0.9944444444444445, 'valid': 0.95, 'test': 0.9666666666666667}}


C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myen

In [17]:
# Liste des valeurs de profondeur maximale à tester
max_depths = [i for i in range(3, 11)]

# Dictionnaire pour stocker les scores de chaque profondeur maximale
scores = {}

# Itère sur chaque profondeur maximale à tester
for max_depth in max_depths:
    # Crée un modèle DecisionTreeClassifier avec la profondeur maximale spécifiée
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

    # Applique la fonction kfold pour effectuer la validation croisée Leave-One-Out (loo)
    # et récupère les scores d'accuracy pour chaque ensemble (train, valid, test)
    scoring = kfold(X, y, model, loo=True)

    # Stocke les scores dans le dictionnaire avec la profondeur maximale comme clé
    scores[str(max_depth)] = scoring

# Affiche les scores pour chaque profondeur maximale
print(scores)

C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myen

{'3': {'train': 0.9747899159663864, 'valid': 1.0, 'test': 0.9666666666666666}, '4': {'train': 0.9747899159663864, 'valid': 1.0, 'test': 0.9666666666666666}, '5': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}, '6': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}, '7': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}, '8': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}, '9': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}, '10': {'train': 0.9915966386554622, 'valid': 1.0, 'test': 0.9666666666666666}}


C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\Leo\anaconda3\envs\myen

In [18]:
# Importe les modules nécessaires
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Divise les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Définit la grille de recherche des hyperparamètres
params = {'max_depth': [i for i in range(3, 11)]}

# Crée un modèle DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

# Initialise un objet GridSearchCV avec le modèle, la grille de paramètres, la mesure de performance ('accuracy') et le nombre de plis pour la validation croisée (cv=5)
clf = GridSearchCV(decision_tree, param_grid=params, scoring='accuracy', cv=5)

# Effectue la recherche sur la grille en ajustant le modèle aux données d'entraînement
clf.fit(X_train, y_train)

# Prédit les étiquettes pour l'ensemble de test avec le meilleur modèle trouvé par la recherche sur la grille
preds = clf.predict(X_test)

# Calcule et affiche le score d'accuracy pour les prédictions sur l'ensemble de test
scoring = metrics.accuracy_score(y_test, preds)
print(scoring)

1.0
